In [8]:
import sejong_corpus_cleaner

In [9]:
import re
import kss
from bs4 import BeautifulSoup

In [10]:
from sqlalchemy import Column, Integer, String, create_engine, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

In [11]:
from sqlalchemy.orm import sessionmaker

In [12]:
from google.cloud import translate_v2 as translate

In [13]:
REGEX_SPECIAL = '[-~!=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》∎○■★+△▴]'

In [14]:
written_paths = sejong_corpus_cleaner.get_data_paths(corpus_types='written')
colloquial_paths = sejong_corpus_cleaner.get_data_paths(corpus_types='colloquial')

In [15]:
engine = create_engine('sqlite:///corpus.db', echo=False)

In [16]:
base = declarative_base()

In [17]:
class SejongCorpus(base):
    __tablename__ = 'SEJONG_CORPUS'
    
    pk = Column('pk', Integer, primary_key=True, autoincrement=True)
    ko_text = Column('ko_text', String, nullable=True)
    thai_text = Column('thai_text', String, nullable=True)
    file_name = Column('file_name', String, nullable=True)
    sent_count = Column('sent_count', Integer, nullable=True)
    cumsum_count = Column('cumsum_count', Integer, nullable=True)
    
    def __repr__(self):
        return 'pk:{}, ko_text:{}, thai_text:{}'.format(self.pk, self.ko_text, self.thai_text)

In [34]:
base.metadata.create_all(bind=engine)

In [34]:
Session = sessionmaker(bind=engine)
session = Session()

In [19]:
client = translate.Client()

In [7]:
client.from_service_account_json('/Users/jaehyunlee/civil_translation/Data/My First Project-22345423bbbe.json')

In [ ]:
sentence_count = 0
complete_file_list = list()

In [37]:
# 문어체
for written in written_paths:
    with open(written, 'r', encoding='utf-8') as f:
        filename = f.name.split('/')[-1]
        print('file name : {}'.format(filename))
        
        if filename not in complete_file_list:
            soup = BeautifulSoup(f.read(), 'html.parser')
            p_tags = soup.find_all('p')

            for p in p_tags:
                p_text = p.text
                p_text = re.sub(REGEX_SPECIAL, '', p_text)

    #             print('sentence line count : {}'.format(len(kss.split_sentences(p_text))))
                for sent in kss.split_sentences(p_text):
                    previous_obj = session.query(SejongCorpus.ko_text).filter_by(ko_text=sent)
                    sent_count = 0

                    if previous_obj.count() == 0:
                        # 번역 진행
                        th_text = client.translate(sent, target_language='th')['translatedText']

                        # 번역한 문장 건수 +1
                        sent_count += 1
                        corpus_obj = SejongCorpus(ko_text=sent, 
                                                  thai_text=th_text, 
                                                  file_name=f.name.split('/')[-1], 
                                                  sent_count=sent_count,
                                                  cumsum_count=len(kss.split_sentences(p_text)))

                        session.add(corpus_obj)
                        session.commit()
                        # 전체건수 + 1
                        sentence_count += 1
                        print('cumsum total count : {}'.format(sentence_count))
            complete_file_list.append(filename)
else:
    print('sentence total count : {}'.format(sentence_count))

file name : BA90A067.txt
file name : BA90A073.txt
file name : BA90A083.txt
file name : BA90A089.txt
file name : BA90A098.txt
file name : BA90A107.txt
file name : BA91A066.txt
file name : BA91A079.txt
file name : BA91A084.txt
file name : BA91A091.txt
file name : BA91A099.txt
file name : BA91A108.txt
file name : BA92A012.txt
file name : BA93A004.txt
file name : BA93A070.txt
file name : BA93A082.txt
file name : BA93A092.txt
file name : BA93B103.txt
file name : BA94L050.txt
file name : BA94L051.txt
file name : BA94L053.txt
file name : BA94L054.txt
file name : BB93B003.txt
file name : BB94D021.txt
file name : BB9XF012.txt
file name : BB9XF013.txt
file name : BB9XF018.txt
file name : BB9XF023.txt
file name : BB9XF024.txt
file name : BB9XZ025.txt
file name : BB9XZ026.txt
file name : BB9XZ027.txt
file name : BCCX0014.txt
file name : BDXX0007.txt
file name : BDXX0008.txt
file name : BDXX0010.txt
file name : BEXX0003.txt
file name : BEXX0004.txt
file name : BEXX0011.txt
file name : BEXX0012.txt


cumsum total count : 359453
cumsum total count : 359454
cumsum total count : 359455
cumsum total count : 359456
cumsum total count : 359457
cumsum total count : 359458
cumsum total count : 359459
cumsum total count : 359460
cumsum total count : 359461
cumsum total count : 359462
cumsum total count : 359463
cumsum total count : 359464
cumsum total count : 359465
cumsum total count : 359466
cumsum total count : 359467
cumsum total count : 359468
cumsum total count : 359469
cumsum total count : 359470
cumsum total count : 359471
cumsum total count : 359472
cumsum total count : 359473
cumsum total count : 359474
cumsum total count : 359475
cumsum total count : 359476
cumsum total count : 359477
cumsum total count : 359478
cumsum total count : 359479
cumsum total count : 359480
cumsum total count : 359481
cumsum total count : 359482
cumsum total count : 359483
cumsum total count : 359484
cumsum total count : 359485
cumsum total count : 359486
cumsum total count : 359487
cumsum total count :

cumsum total count : 359746
cumsum total count : 359747
cumsum total count : 359748
cumsum total count : 359749
cumsum total count : 359750
cumsum total count : 359751
cumsum total count : 359752
cumsum total count : 359753
cumsum total count : 359754
cumsum total count : 359755
cumsum total count : 359756
cumsum total count : 359757
cumsum total count : 359758
cumsum total count : 359759
cumsum total count : 359760
cumsum total count : 359761
cumsum total count : 359762
cumsum total count : 359763
cumsum total count : 359764
cumsum total count : 359765
cumsum total count : 359766
cumsum total count : 359767
cumsum total count : 359768
cumsum total count : 359769
cumsum total count : 359770
cumsum total count : 359771
cumsum total count : 359772
cumsum total count : 359773
cumsum total count : 359774
cumsum total count : 359775
cumsum total count : 359776
cumsum total count : 359777
cumsum total count : 359778
cumsum total count : 359779
cumsum total count : 359780
cumsum total count :

cumsum total count : 360039
cumsum total count : 360040
cumsum total count : 360041
cumsum total count : 360042
cumsum total count : 360043
cumsum total count : 360044
cumsum total count : 360045
cumsum total count : 360046
cumsum total count : 360047
cumsum total count : 360048
cumsum total count : 360049
cumsum total count : 360050
cumsum total count : 360051
cumsum total count : 360052
cumsum total count : 360053
cumsum total count : 360054
cumsum total count : 360055
cumsum total count : 360056
cumsum total count : 360057
cumsum total count : 360058
cumsum total count : 360059
cumsum total count : 360060
cumsum total count : 360061
cumsum total count : 360062
cumsum total count : 360063
cumsum total count : 360064
cumsum total count : 360065
cumsum total count : 360066
cumsum total count : 360067
cumsum total count : 360068
cumsum total count : 360069
cumsum total count : 360070
cumsum total count : 360071
cumsum total count : 360072
cumsum total count : 360073
cumsum total count :

cumsum total count : 360332
cumsum total count : 360333
cumsum total count : 360334
cumsum total count : 360335
cumsum total count : 360336
cumsum total count : 360337
cumsum total count : 360338
cumsum total count : 360339
cumsum total count : 360340
cumsum total count : 360341
cumsum total count : 360342
cumsum total count : 360343
cumsum total count : 360344
cumsum total count : 360345
cumsum total count : 360346
cumsum total count : 360347
cumsum total count : 360348
cumsum total count : 360349
cumsum total count : 360350
cumsum total count : 360351
cumsum total count : 360352
cumsum total count : 360353
cumsum total count : 360354
cumsum total count : 360355
cumsum total count : 360356
cumsum total count : 360357
cumsum total count : 360358
cumsum total count : 360359
cumsum total count : 360360
cumsum total count : 360361
cumsum total count : 360362
cumsum total count : 360363
cumsum total count : 360364
cumsum total count : 360365
cumsum total count : 360366
cumsum total count :

...

cumsum total count : 396652
cumsum total count : 396653
cumsum total count : 396654
cumsum total count : 396655
cumsum total count : 396656
cumsum total count : 396657
cumsum total count : 396658
cumsum total count : 396659
cumsum total count : 396660
cumsum total count : 396661
cumsum total count : 396662
cumsum total count : 396663
cumsum total count : 396664
cumsum total count : 396665
cumsum total count : 396666
cumsum total count : 396667
cumsum total count : 396668
cumsum total count : 396669
cumsum total count : 396670
cumsum total count : 396671
cumsum total count : 396672
cumsum total count : 396673
cumsum total count : 396674
cumsum total count : 396675
cumsum total count : 396676
cumsum total count : 396677
cumsum total count : 396678
cumsum total count : 396679
cumsum total count : 396680
cumsum total count : 396681
cumsum total count : 396682
cumsum total count : 396683
cumsum total count : 396684
cumsum total count : 396685
cumsum total count : 396686
cumsum total count :

cumsum total count : 396945
cumsum total count : 396946
cumsum total count : 396947
cumsum total count : 396948
cumsum total count : 396949
cumsum total count : 396950
cumsum total count : 396951
cumsum total count : 396952
cumsum total count : 396953
cumsum total count : 396954
cumsum total count : 396955
cumsum total count : 396956
cumsum total count : 396957
cumsum total count : 396958
cumsum total count : 396959
cumsum total count : 396960
cumsum total count : 396961
cumsum total count : 396962
cumsum total count : 396963
cumsum total count : 396964
cumsum total count : 396965
cumsum total count : 396966
cumsum total count : 396967
cumsum total count : 396968
cumsum total count : 396969
cumsum total count : 396970
cumsum total count : 396971
cumsum total count : 396972
cumsum total count : 396973
cumsum total count : 396974
cumsum total count : 396975
cumsum total count : 396976
cumsum total count : 396977
cumsum total count : 396978
cumsum total count : 396979
cumsum total count :

cumsum total count : 397238
cumsum total count : 397239
cumsum total count : 397240
cumsum total count : 397241
cumsum total count : 397242
cumsum total count : 397243
cumsum total count : 397244
cumsum total count : 397245
cumsum total count : 397246
cumsum total count : 397247
cumsum total count : 397248
cumsum total count : 397249
cumsum total count : 397250
cumsum total count : 397251
cumsum total count : 397252
cumsum total count : 397253
cumsum total count : 397254
cumsum total count : 397255
cumsum total count : 397256
cumsum total count : 397257
cumsum total count : 397258
cumsum total count : 397259
cumsum total count : 397260
cumsum total count : 397261
cumsum total count : 397262
cumsum total count : 397263
cumsum total count : 397264
cumsum total count : 397265
cumsum total count : 397266
cumsum total count : 397267
cumsum total count : 397268
cumsum total count : 397269
cumsum total count : 397270
cumsum total count : 397271
cumsum total count : 397272
cumsum total count :

cumsum total count : 397531
cumsum total count : 397532
cumsum total count : 397533
cumsum total count : 397534
cumsum total count : 397535
cumsum total count : 397536
cumsum total count : 397537
cumsum total count : 397538
cumsum total count : 397539
cumsum total count : 397540
cumsum total count : 397541
cumsum total count : 397542
cumsum total count : 397543
cumsum total count : 397544
cumsum total count : 397545
cumsum total count : 397546
cumsum total count : 397547
cumsum total count : 397548
cumsum total count : 397549
cumsum total count : 397550
cumsum total count : 397551
cumsum total count : 397552
cumsum total count : 397553
cumsum total count : 397554
cumsum total count : 397555
cumsum total count : 397556
cumsum total count : 397557
cumsum total count : 397558
cumsum total count : 397559
cumsum total count : 397560
cumsum total count : 397561
cumsum total count : 397562
cumsum total count : 397563
cumsum total count : 397564
cumsum total count : 397565
cumsum total count :

KeyboardInterrupt: 

In [26]:
complete_file_list # 4474347

['BA90A067.txt',
 'BA90A073.txt',
 'BA90A083.txt',
 'BA90A089.txt',
 'BA90A098.txt',
 'BA90A107.txt',
 'BA91A066.txt',
 'BA91A079.txt',
 'BA91A084.txt',
 'BA91A091.txt',
 'BA91A099.txt',
 'BA91A108.txt',
 'BA92A012.txt',
 'BA93A004.txt',
 'BA93A070.txt',
 'BA93A082.txt',
 'BA93A092.txt',
 'BA93B103.txt',
 'BA94L050.txt',
 'BA94L051.txt',
 'BA94L053.txt',
 'BA94L054.txt',
 'BB93B003.txt',
 'BB94D021.txt',
 'BB9XF012.txt',
 'BB9XF013.txt',
 'BB9XF018.txt',
 'BB9XF023.txt',
 'BB9XF024.txt',
 'BB9XZ025.txt',
 'BB9XZ026.txt',
 'BB9XZ027.txt',
 'BCCX0014.txt',
 'BDXX0007.txt',
 'BDXX0008.txt',
 'BDXX0010.txt',
 'BEXX0003.txt',
 'BEXX0004.txt',
 'BEXX0011.txt',
 'BEXX0012.txt',
 'BEXX0013.txt',
 'BEXX0014.txt',
 'BEXX0028.txt',
 'BGAA0001.txt',
 'BGAA0164.txt',
 'BGAE0200.txt',
 'BGBZ0073.txt',
 'BGEO0077.txt',
 'BGEO0292.txt',
 'BGEO0320.txt',
 'BGGO0098.txt',
 'BGGO0358.txt',
 'BGHO0107.txt',
 'BGHO0127.txt',
 'BGHO0409.txt',
 'BGHO0411.txt',
 'BGHO0431.txt',
 'BGHO0437.txt',
 'BGXX0035.txt

In [ ]:
# 구어체
for colloquial in colloquial_paths:
    with open(colloquial, 'r', encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        p_tags = soup.find_all('p')

        for p in p_tags:
            p_text = p.text
            p_text = re.sub(REGEX_SPECIAL, '', p_text)
            print(p_text)
            
            for sent in kss.split_sentences(p_text):
                th_text = client.translate(sent, target_language='th')['translatedText']
                corpus_obj = SejongCorpus(ko_text=sent, thai_text=th_text)
                session.add(corpus_obj)
                session.commit()